In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(directory['py'])
import techdata_to_tree, sys, ShockFunction
import abatement_EOP_new as abatement
os.chdir(directory['curr'])

*Specify settings:*

In [2]:
name = 'A4'
# catalogue = 'dors_2'
catalogue = 'C2'
f = {'tech': directory['data']+f"/techdata_{catalogue}.xlsx",
     'other': directory['data']+f"/Othdata_{catalogue}.xlsx",
     'IOdata': directory['data']+f"/IOdata_{catalogue}.xlsx"}
s = ['inputdisp','endofpipe','inputprices']
m = ['ID','EOP'] 
e = {'M': 'vars','M_sets': 'sets'} # dictionary used to load emissions data
other_trees = {'ID_Y': {'s': 'Y', 'read_data': {'Y': 'vars'}, 'Q2P': 'Q2P'}} # the key = tree name, s = sheet that specifies tree structure, read_data (if not None) specifies how to load data.

In [3]:
type_f = {'ID_EC': 'CES_norm',
          'ID_CU': 'MNL',
          'ID_TU': 'CET_norm',
          'ID_TX': 'CES',
          'ID_BU': 'linear_out',
          'ID_BX': 'CES',
          'EOP_CU': 'MNL',
          'EOP_TU': 'CET_norm',
          'EOP_TX': 'CES'}

*Set up nesting tree:*

In [4]:
%run ATree.ipynb

In [5]:
dbs = {}

## 1: Model without EOP

*Initialize without EOP:*

In [6]:
gm = abatement.abate(nts=nts,tech=tech,work_folder=directory['work'],use_EOP=False,**{'data_folder':directory['gams'],'name': name})
gm.add_aliases([('n','nnnn'),('n','nnnnn')])
gm.model.database.update_all_sets(clean_up=False) 

*Provide initial values:*

In [7]:
gm.initialize_variables_leontief()
gm.initialize_variables(**{'check_variables': True})

*Add some extra options:*

In [8]:
with open(os.getcwd()+'\\conopt_ExtraPivot.opt',"r") as file:
    kwargs_mi = {'opt_file': file.read()}

### 1.1: Solve with leontief values:

In [9]:
gm.write_and_run(name='ID',kwargs_mi=kwargs_mi)
dbs['ID'] = gm.model_instances['ID'].out_db

*Merge into starting values:*

In [10]:
gm.model.database.merge_dbs(gm.model.database,dbs['ID'],'second')

*Add new target smoothing values:*

In [11]:
db_star = DataBase.GPM_database()
db_star['eta'] = pd.Series(-5, index = gm.get('eta').index, name = 'eta')
db_star['sigma'] = pd.Series(5, index = gm.get('kno_ID_CU'),name='sigma').combine_first(
                   pd.Series(0.5, index = gm.get('kno_ID_BX'),name='sigma')).combine_first(
                   pd.Series(0.5, index = gm.get('kno_ID_Y'), name = 'sigma'))
db_star['mu'] = pd.Series(1, index = gm.get('map_ID_CU'),name='mu')

*Solve smoothly:*

In [12]:
gm.model_instances['ID'].solve_sneakily(db_star=db_star,kwargs_shock={'n_steps':100})

{'Modelstat': 16.0, 'Solvestat': 1.0}

*Write back data:*

In [13]:
gm.model.database.merge_dbs(gm.model.database,gm.model_instances['ID'].out_db,'second')

### 1.2: Calibrate

In [14]:
gm.reset_settings()
gm.setstate('ID_calibrate',init=False)

*Load IO data:*

In [15]:
db = gm.add_calib_data(f["IOdata"], currapp=False)

*Percentage deviation from starting point:*

*test run:*

In [16]:
gm.write_and_run(name='ID_calibrate',kwargs_init={'check_variables':True},options_run={'output':sys.stdout},overwrite=True,kwargs_mi=kwargs_mi)
# gm.write_and_run(name='ID_calibrate',kwargs_init={'check_variables':True},overwrite=True,kwargs_mi=kwargs_mi)

--- Job CollectAndRun.gmy Start 09/16/21 15:55:49 29.1.0 rbb4180b WEX-WEI x86 64bit/MS Windows
--- GAMS Parameters defined
    LP CONOPT4
    MIP CPLEX
    RMIP CONOPT4
    NLP CONOPT4
    MCP PATH
    MPEC NLPEC
    RMPEC NLPEC
    CNS CONOPT4
    DNLP CONOPT4
    RMINLP CONOPT4
    MINLP DICOPT
    QCP CONOPT4
    MIQCP SBB
    RMIQCP CONOPT4
    EMP JAMS
    Input C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\CollectAndRun.gmy
    Output C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gjo3.lst
    ScrDir C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\225c\
    SysDir C:\GAMS\win64\29.1\
    CurDir C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\
    LogOption 3
    LogFile C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gjo3.log
    GDX C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gdb3.gdx
    --ID_0 ID_0
    --shock shock
GAMS 29.1.0   Copyright (C) 1987-2019 GAMS Development. All rights reserved
Licensee: Medium MUD - 

*Sneaky solve:*

In [17]:
gm.model_instances['ID'].out_db.get('currapp')

n   nn   
EH  ID_t2    0.000001
    ID_t1    0.000031
Name: currapp, dtype: float64

In [18]:
gm.model_instances['ID_calibrate'].solve_sneakily(db_star=db,options_run={'output':sys.stdout},kwargs_shock={'n_steps':10, 'solvetext': gm.model.settings.get_conf('solve')})

--- Job CollectAndRun.gmy Start 09/16/21 15:55:50 29.1.0 rbb4180b WEX-WEI x86 64bit/MS Windows
--- GAMS Parameters defined
    LP CONOPT4
    MIP CPLEX
    RMIP CONOPT4
    NLP CONOPT4
    MCP PATH
    MPEC NLPEC
    RMPEC NLPEC
    CNS CONOPT4
    DNLP CONOPT4
    RMINLP CONOPT4
    MINLP DICOPT
    QCP CONOPT4
    MIQCP SBB
    RMIQCP CONOPT4
    EMP JAMS
    Input C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\CollectAndRun.gmy
    Output C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gjo4.lst
    Save C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gcp1.g0?
    ScrDir C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\225c\
    SysDir C:\GAMS\win64\29.1\
    CurDir C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\
    LogOption 3
    LogFile C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gjo4.log
    GDX C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gdb4.gdx
    --ID_0 ID_0
    --shock shock
GAMS 29.1.0   Copyr

GamsExceptionExecution: GAMS return code not 0 (3), check C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gjo5.lst for more details